In [ ]:
import pandas as pd
import os
import openpyxl

아래 행안부 데이터에서 원하는 자료 다운후 한 파일에 모으기

https://www.localdata.kr/devcenter/dataDown.do?menuNo=20001

In [ ]:
path = #'파일위치'
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]

# 데이터프레임 리미트 해제
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

In [ ]:
#폴더안 CSV파일 합쳐 한 데이터 프레임으로 만들기
df = pd.DataFrame()
for i in file_list_py:
    data = pd.read_csv(path + i, encoding = 'cp949')
    df = pd.concat([df,data])
    
df = df.reset_index(drop = True)
#df.head()
pd.options.display.float_format = '{:.0f}'.format #소수점 제외(위도, 경도 필요할시 0갯수 늘려주기)


프렌차이즈 카페 알아보기 위해 공정거래위원회 가매사업정보 시스템 확인
https://franchise.ftc.go.kr/mnu/00014/program/firHope/view.do
등록년도: 2020, 비교 항목: 브랜드별 and 브랜드 개요
업종 : 외식 and 커피

In [ ]:
df_franchise = pd.read_excel(#'파일위치/2021 프렌차이즈 카페.xlsx', sheet_name= 1)
df_franchise_1 = df_franchise[df_franchise["가맹점수"]>=10] #가맹점수 10개 이상인 프렌차이즈만 불러오기
cafe_list = list(df_franchise_1['브랜드'])
#cafe_list #카페 목록 리스트로 변환
#cafe_list

In [ ]:
df_brand_all = pd.DataFrame(columns=['1. 전체 = 폐업 + 개업 포함', '2. 폐업', '3. 영업', "4. 2020년과 2021년에 문을 닫은 가게",
                               "5. 2020년 이전에 닫은 가게", "6. 2020년 이후에 오픈한 매장수", "7. 2020년 이후에 오픈하고 2020년 이후 폐점한 매장수"])
for i in cafe_list:
    #프렌차이즈 폐업수와 개업수, 현재 매장을 알아보자
    df_cafe = df[df["사업장명"].str.contains( i , na = False) ] # 전체 이디야 = 폐업 + 개업 포함 
    df_cafe = df_cafe[df_cafe["업태구분명"].isin(['까페', '커피숍', '다방'])]
    print(i)
    #print('1. 전체 = 폐업 + 개업 포함')
    #print(len(df_cafe))

    df_cafe["폐업일자"] = df_cafe["폐업일자"].astype(float) #폐업일자 숫자로 전환

    df_cafe_1 = df_cafe[df_cafe["상세영업상태명"].isin(['폐업'])] #폐업
    #print("2. 폐업" )
    #print(len(df_cafe_1))
    df_cafe_2 = df_cafe[df_cafe["상세영업상태명"].isin(['영업'])] #영업
    #print("3. 영업" )
    #print(len(df_cafe_2))

    #2020년과 2021년에 문을 닫은 가게
    df_cafe_4 = df_cafe[df_cafe["상세영업상태명"].isin(['폐업']) & (df_cafe["폐업일자"] >= 20200101)] 
    #print("4. 2020년과 2021년에 문을 닫은 가게" )
    #print(len(df_cafe_4))
    #2020년 이전에 닫은 가게 
    df_cafe_5 = df_cafe[df_cafe["상세영업상태명"].isin(['폐업']) & (df_cafe["폐업일자"] < 20200101) ] 
    #print("5. 2020년 이전에 닫은 가게")
    #print(len(df_cafe_5))


    #폐업점 엑셀로 저장
    #df_ediya_1.to_excel(excel_writer='이디야 폐업점.xlsx')

    #2020년 이후에 오픈한 매장수 
    df_cafe_7 = df_cafe_2[df_cafe_2["인허가일자"] >= 20200101]
    #print("6. 2020년 이후에 오픈한 매장수 ")
    #print(len(df_cafe_7))

    #2020년 이후에 오픈하고 2020년 이후 폐점한 매장수
    df_cafe_8 = df_cafe[(df_cafe["인허가일자"] >= 20201001) & (df_cafe["폐업일자"] >= 20201001)]
    #print("7. 2020년 이후에 오픈하고 2020년 이후 폐점한 매장수 ")
    #print(len(df_cafe_8))
    print("-------------------------------------------------------------------------")
    
    df_list =  [len(df_cafe)] + [len(df_cafe_1)]+[len(df_cafe_2)]+ [len(df_cafe_4)]+ [len(df_cafe_5)]+ [len(df_cafe_7)]+ [len(df_cafe_8)]
    df_brand_all.loc[i] = df_list

df_brand_all.to_excel(excel_writer='2021 카페 프렌차이즈 개폐업 현황.xlsx')

In [ ]:
#영업중인 점포 찾아보기
df_brand_all.sort_values(by=['3. 영업'], axis=0, ascending=False)